In [214]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: Qt5Agg


In [215]:
problem, setting, res = pickle.load(open('full_res_nsga3.pkl', 'rb'))

In [216]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
x_Ptx_dBm[~x_pass] = np.NaN
x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)
#x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

f_pointing = res.F[:, 3] # s
f_energy = res.F[:,2] / 1e3 # Kilo Joule
f_latency = res.F[:,1]   # s
f_throughput = ((res.F[:,0] * -1) / 1e9)   # Gigabit

In [217]:
points = []
F_norm = res.F / np.max(res.F, axis=0)

# Max throughput point
ind = np.argmax(f_throughput)
p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
points.append({'ind': ind, 'p': p, 'color': 'r', 'marker': '.', 'size': 10, 'label': 'Max T\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# Max E, minimum P point
ref = np.array([1, 0])
ind = np.argmin(np.linalg.norm(F_norm[:,2:4] - ref, axis = 1))
p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
points.append({'ind': ind, 'p': p, 'color': 'g', 'marker': '.', 'size': 10, 'label': 'Max E, Min P\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# 1000 GBit, min Energy
ref = np.array([-1000e9, 0])
ind = np.argmin(np.linalg.norm((res.F[:,[0,2]] - ref) / np.max(res.F[:,[0,2]] - ref, axis = 0), axis = 1))
p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
points.append({'ind': ind, 'p': p, 'color': 'b', 'marker': '.', 'size': 10, 'label': 'T = 1000 GBit, Min E\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# 3600 s, min Energy
# ref = np.array([3600, 0])
# ind = np.argmin(np.linalg.norm((res.F[:,[1,2]] - ref) / np.max(res.F[:,[0,2]] - ref, axis = 0), axis = 1))
# p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'y', 'marker': '.', 'size': 10, 'label': 'L = 3600 s, Min E\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# 3600 s, min Energy
# ref = np.array([3600, 0])
# ind = np.argmin(np.linalg.norm((res.F[:,[1,2]] - ref) / np.max(res.F[:,[0,2]] - ref, axis = 0), axis = 1))
# p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'y', 'marker': '.', 'size': 10, 'label': 'L = 3600 s, Min E\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# # Reference points
# ref_points = setting['algorithm'].survival.ref_points / np.array([-1e9, 1e3, 1])
# points.append({'ind': ind, 'p': ref_points[0,[1,2,0]], 'color': 'r', 'marker': 'x', 'size': 15, 'label': 'Target 1\n(%d Gbit, %d kJ, %d s)' % (ref_points[0,0], ref_points[0,1], ref_points[0,2])})
# points.append({'ind': ind, 'p': ref_points[1,[1,2,0]], 'color': 'g', 'marker': 'x', 'size': 15, 'label': 'Target 2\n(%d Gbit, %d kJ, %d s)' % (ref_points[1,0], ref_points[1,1], ref_points[1,2])})

In [218]:
T_lim = (0, 2000)
L_lim = (0, 5000)
E_lim = (0, 100)
P_lim = (0, 2100)

# offset 0 limit with 10%
perc = 0.025
T_lim = np.array(T_lim) - np.array([perc * (T_lim[1] - T_lim[0]), 0])
L_lim = np.array(L_lim) - np.array([perc * (L_lim[1] - L_lim[0]), 0])
E_lim = np.array(E_lim) - np.array([perc * (E_lim[1] - E_lim[0]), 0])
P_lim = np.array(P_lim) - np.array([perc * (P_lim[1] - P_lim[0]), 0])

In [219]:
plt.close('all')

fig = plt.figure(figsize=(3.2*3, 2.4*3))
axs = fig.subplots(3,3, gridspec_kw={'height_ratios': [1, 1, 1]})
axs = axs.flatten()

ax = axs[0]
ax.grid(True)
ax.scatter(f_energy, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][2], point['p'][0],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim(E_lim)
ax.set_ylim(T_lim)
ax.set_axisbelow(True)

ax = axs[1]
ax.grid(True)
ax.scatter(f_pointing, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][3], point['p'][0],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Pointing [s]")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim(P_lim)
ax.set_ylim(T_lim)
ax.set_axisbelow(True)

ax = axs[2]
ax.grid(True)
ax.scatter(f_latency, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][1], point['p'][0],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

# ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
# ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
ax.set_xlabel("Avg. Latency [s]")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim(L_lim)
ax.set_ylim(T_lim)
ax.set_axisbelow(True)

ax = axs[3]
ax.grid(True)
ax.scatter(f_energy, f_latency, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][2], point['p'][1],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Avg. Latency [s]")
ax.set_xlim(E_lim)
ax.set_ylim(L_lim)
ax.set_axisbelow(True)

ax = axs[4]
ax.grid(True)
ax.scatter(f_pointing, f_latency, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][3], point['p'][1],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Pointing [s]")
ax.set_ylabel("Avg. Latency [s]")
ax.set_xlim(P_lim)
ax.set_ylim(L_lim)
ax.set_axisbelow(True)

ax = axs[5]
ax.grid(True)
ax.scatter(f_energy, f_pointing, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][2], point['p'][3],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

# ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
# ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Pointing [s]")
ax.set_xlim(E_lim)
ax.set_ylim(P_lim)
ax.set_axisbelow(True)

axs[6].set_axis_off()
axs[7].set_axis_off()
ax = axs[8]
for point in points:
    ax.scatter(np.nan, np.nan,
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.legend(fontsize=8)
ax.set_axis_off()

plt.tight_layout()


import matplotlib
x,y = np.array([[2/3 - 0.005, 2/3-0.005], [1/3-0.03, 1]])
hline = matplotlib.lines.Line2D(x, y, lw=0.75, color='k', alpha=0.5)
hline.set_clip_on(False)
x,y = np.array([[0, 1], [2/3-0.021, 2/3-0.021]])
vline = matplotlib.lines.Line2D(x, y, lw=0.75, color='k', alpha=0.5)
vline.set_clip_on(False)
ax_over = plt.axes([0,0,1,1], facecolor=(1,1,1,0))
ax_over.add_line(hline)
ax_over.add_line(vline)

# autoAxis = sub1.axis()
# rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2)
# rec = sub1.add_patch(rec)
# rec.set_clip_on(False)

fig.savefig('D:/git/thesis_report_ae/figures/full_pareto_six_%s.pdf' % setting['label'])

In [220]:
# Plot a legend
fig, ax = plt.subplots(figsize=(3.2, 2.4))

for point in points:
    ax.scatter(np.nan, np.nan,
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_axis_off()
PlotUtil.apply_report_formatting()
ax.legend(fontsize=8)
fig.savefig('D:/git/thesis_report_ae/figures/full_pareto_%s_leg.pdf' % setting['label'])
